# Implementación de un Generador de Batches para Procesamiento de Datos con Xarray

En este notebook, implementaremos paso a paso un **Generador de Batches** eficiente. Este generador es fundamental para preparar grandes conjuntos de datos para tareas de aprendizaje automático, permitiendo manejar de manera efectiva la memoria y optimizar el proceso de entrenamiento de modelos.

## Objetivos

1. **División de Datos en Muestras**: Crear ventanas deslizantes sobre el eje temporal del dataset para generar muestras que serán utilizadas en el entrenamiento.
2. **Barajar Muestras**: Implementar una funcionalidad para barajar las muestras, asegurando que el modelo no aprenda patrones específicos del orden de los datos.
3. **Generación de Batches**: Agrupar las muestras en batches de tamaño definido, facilitando el procesamiento en paralelo y mejorando la eficiencia computacional.
4. **Manejo de Batches Residuales**: Aplicar padding al último batch si no completa el tamaño establecido, garantizando consistencia en el tamaño de los batches.
5. **Verificación y Validación**: Realizar comprobaciones para asegurar que cada paso se ha ejecutado correctamente y que los batches generados cumplen con los requisitos establecidos.

## Pasos a Seguir

### 1. Carga y Exploración del Dataset

Antes de comenzar con la generación de batches, es esencial cargar y explorar el dataset que utilizaremos. Esto nos permitirá entender la estructura de los datos y planificar adecuadamente la creación de muestras y batches.

```python
import xarray as xr
import numpy as np

# Cargar el dataset
dataset = xr.open_dataset('ruta/al/dataset.nc')

# Explorar las dimensiones y variables
print(dataset)


## Carga de Datos

In [ ]:
import random
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset
import xarray as xr
from tqdm import tqdm



dataset = xr.open_dataset("D://Aaron//cmems_mod_glo_phy_my_0.083deg_P1D-m_6years_thetao_v3.nc",chunks={"time": 90})
dataset

<xarray.Dataset> Size: 32GB
Dimensions:    (depth: 49, latitude: 180, longitude: 180, time: 2558)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 20kB 2014-01-01 2014-01-02 ... 2021-01-01
Data variables:
    thetao     (time, depth, latitude, longitude) float64 32GB dask.array<chunksize=(90, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.3.3

## 1.Generación de Muestras mediante Ventanas Deslizantes

Utilizaremos una ventana deslizante sobre el eje temporal para dividir el dataset en muestras de tamaño fijo (sample_size). Este enfoque es común en tareas de series temporales donde cada muestra captura una secuencia temporal específica.


In [38]:
# Generar ventanas deslizantes sobre el eje temporal del dataset
window_size = 3
windows = [slice(i, i + window_size) for i in range(0, len(dataset.time) - window_size + 1)]

samples = []
for w in windows:
    samples.append(dataset.isel(time=w))

In [39]:
# Comprobar la cantidad de muestras generadas
print(f"Total de muestras generadas: {len(samples)}")

Total de muestras generadas: 2556


Se generaron 2556 muestras, porque el tamaño de la ventana es 3, y el número total de ventanas deslizantes posibles es `len(dataset.time) - window_size + 1`.

In [41]:
# Generar ventanas deslizantes sobre el eje temporal del dataset
window_size = 10
windows = [slice(i, i + window_size) for i in range(0, len(dataset.time) - window_size + 1)]

samples = []
for w in windows:
    samples.append(dataset.isel(time=w))

In [40]:
# Comprobar la cantidad de muestras generadas
print(f"Total de muestras generadas: {len(samples)}")

Total de muestras generadas: 2556


A pesar de cambiar el tamaño de la ventana a 10, 
el número de ventanas sigue siendo 2556, lo cual no es el comportamiento esperado. Esto podría indicar que `dataset.time` no cambió entre las pruebas.

In [10]:
def generate_sliding_windows(dataset, window_size):
    """
    Genera ventanas deslizantes sobre el eje temporal del dataset.

    Args:
        dataset (xr.Dataset): El conjunto de datos que contiene las variables oceánicas.
        window_size (int): Tamaño de la ventana deslizante.

    Returns:
        list: Lista de ventanas deslizantes del dataset.
    """
    windows = [slice(i, i + window_size) for i in range(0, len(dataset.time) - window_size + 1)]
    samples = [dataset.isel(time=w) for w in windows]
    return samples

# Uso de la función
window_size = 3
samples = generate_sliding_windows(dataset, window_size)
samples[0]
samples[1]

<xarray.Dataset> Size: 38MB
Dimensions:    (depth: 49, latitude: 180, longitude: 180, time: 3)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 24B 2014-01-02 2014-01-03 2014-01-04
Data variables:
    thetao     (time, depth, latitude, longitude) float64 38MB dask.array<chunksize=(3, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.3.3

## 2.Barajar las Muestras

Para evitar que el modelo aprenda patrones específicos del orden de los datos, es recomendable barajar las muestras antes de agruparlas en batches. Implementaremos una función que realice esta barajado de manera aleatoria.

In [11]:
samples = generate_sliding_windows(dataset, window_size)
len(samples)

2556

In [12]:
samples_copy = samples.copy()
np.random.shuffle(samples_copy)
samples_random = samples_copy

In [42]:
# Barajar las muestras
np.random.shuffle(samples)

# Verificar el barajado
print("Primeras 3 muestras después del barajado:")
for muestra in samples[:3]:
    print(muestra.time.values)


Primeras 3 muestras después del barajado:
['2014-02-15T00:00:00.000000000' '2014-02-16T00:00:00.000000000'
 '2014-02-17T00:00:00.000000000' '2014-02-18T00:00:00.000000000'
 '2014-02-19T00:00:00.000000000' '2014-02-20T00:00:00.000000000'
 '2014-02-21T00:00:00.000000000' '2014-02-22T00:00:00.000000000'
 '2014-02-23T00:00:00.000000000' '2014-02-24T00:00:00.000000000']
['2014-08-03T00:00:00.000000000' '2014-08-04T00:00:00.000000000'
 '2014-08-05T00:00:00.000000000' '2014-08-06T00:00:00.000000000'
 '2014-08-07T00:00:00.000000000' '2014-08-08T00:00:00.000000000'
 '2014-08-09T00:00:00.000000000' '2014-08-10T00:00:00.000000000'
 '2014-08-11T00:00:00.000000000' '2014-08-12T00:00:00.000000000']
['2019-07-28T00:00:00.000000000' '2019-07-29T00:00:00.000000000'
 '2019-07-30T00:00:00.000000000' '2019-07-31T00:00:00.000000000'
 '2019-08-01T00:00:00.000000000' '2019-08-02T00:00:00.000000000'
 '2019-08-03T00:00:00.000000000' '2019-08-04T00:00:00.000000000'
 '2019-08-05T00:00:00.000000000' '2019-08-06T0

Los valores de tiempo en las muestras ya no están en orden secuencial, lo que confirma que el barajado se realizó correctamente.

In [14]:
samples_random[0]
samples_random[1]

<xarray.Dataset> Size: 38MB
Dimensions:    (depth: 49, latitude: 180, longitude: 180, time: 3)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 24B 2020-05-08 2020-05-09 2020-05-10
Data variables:
    thetao     (time, depth, latitude, longitude) float64 38MB dask.array<chunksize=(3, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.3.3

In [15]:
def shuffle_samples(samples):
    """
    Copia y baraja una lista de muestras.

    Args:
        samples (list): Lista de muestras a barajar.

    Returns:
        list: Lista de muestras barajadas.
    """
    samples_copy = samples.copy()
    np.random.shuffle(samples_copy)
    return samples_copy

# Uso de la función
samples_random = shuffle_samples(samples)

## 3.Generación de Batches

Agruparemos las muestras barajadas en batches de tamaño definido (batch_size). Esto permite procesar múltiples muestras simultáneamente durante el entrenamiento, mejorando la eficiencia.

In [16]:
batch_size=3
batches = [samples_random[i:i + batch_size] for i in range(0, len(samples_random), batch_size)]

In [20]:
def generate_batches(samples, batch_size):
    """
    Divide una lista de muestras en batches.

    Args:
        samples (list): Lista de muestras.
        batch_size (int): Tamaño de cada batch.

    Returns:
        list: Lista de batches.
    """
    assert batch_size % 2 == 0 or batch_size == 1, "batch_size must be multiple of 2"
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]
    return batches

In [44]:
# Prueba de la función
batch_size = 1
batches = generate_batches(samples_random, batch_size)
print(f"Numero de mini-batches: {len(batches)}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in batches]}")
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in batches]}")

Numero de mini-batches: 2556
Tamaño de cada mini-batch: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [23]:
# Prueba de la función
batch_size = 3
batches = generate_batches(samples_random, batch_size)
print(f"Numero de mini-batches: {len(batches)}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in batches]}")
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in batches]}")

AssertionError: batch_size must be multiple of 2

In [24]:
# Prueba de la función
batch_size = 4
batches = generate_batches(samples_random, batch_size)
print(f"Numero de mini-batches: {len(batches)}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in batches]}")
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in batches]}")

Numero de mini-batches: 639
Tamaño de cada mini-batch: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [26]:
# Prueba de la función
batch_size = 8
batches = generate_batches(samples_random, batch_size)
print(f"Numero de mini-batches: {len(batches)}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in batches]}")
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in batches]}")

Numero de mini-batches: 320
Tamaño de cada mini-batch: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,

In [47]:
# Prueba de la función
batch_size = 16
batches = generate_batches(samples_random, batch_size)
print(f"Numero de mini-batches: {len(batches)}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in batches]}")
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in batches]}")

Numero de mini-batches: 160
Tamaño de cada mini-batch: [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 12]
Tamaño de cada sample: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3


- **Total de muestras generadas:** 2556.  
- **Mini-batches:**
  - Se divide la data en mini-batches con diferentes tamaños:
    - Cuando `batch_size=1`: Total de mini-batches: **2556**.
    - Cuando `batch_size=3`: Ocurrió un error (`AssertionError`), ya que el tamaño del batch debe ser múltiplo de 2 o igual a 1.
    - Cuando `batch_size=4`: Total de mini-batches: **639**.
    - Cuando `batch_size=8`: Total de mini-batches: **320**.
    - Cuando `batch_size=16`: Total de mini-batches: **160**.
- **Tamaño de cada sample en un mini-batch:** Siempre es **3** en todas las pruebas.

- **Error notable:**
  - Para valores de `batch_size` que no cumplen la condición (`múltiplo de 2 o igual a 1`), se lanza un `AssertionError`.




## 4.Manejo de Batches Residuales mediante Padding

Es posible que el último batch no complete el tamaño establecido (batch_size). Para mantener la consistencia en el procesamiento, rellenaremos este batch con muestras vacías (rellenas con ceros)


In [45]:


def pad_batches(batches, batch_size):
    """
    Rellena el último batch con ceros para que tenga el mismo tamaño que los demás.

    Args:
        batches (list): Lista de batches.
        batch_size (int): Tamaño de cada batch.

    Returns:
        list: Lista de batches con el último batch rellenado con ceros si es necesario.
    """
    if len(batches[-1]) < batch_size:
        last_batch = batches[-1]
        while len(last_batch) < batch_size:
            last_batch.append(xr.zeros_like(last_batch[0]))  
        batches[-1] = last_batch  
    return batches


# Prueba de la función
batch_size = 8
batches = generate_batches(samples_random, batch_size)
padded_batches = pad_batches(batches, batch_size)
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in padded_batches]}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in padded_batches]}")




Tamaño de cada sample: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Tamaño de cada m

In [29]:
batch_size = 16
batches = generate_batches(samples_random, batch_size)
padded_batches = pad_batches(batches, batch_size)
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in padded_batches]}")
print(f"Tamaño de cada mini-batch: {[len(batch) for batch in padded_batches]}")

Tamaño de cada sample: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Tamaño de cada mini-batch: [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 

### Comentarios de los Resultados

- **Tamaño de las muestras:** Consistente en **3** en todos los casos (`len(batch[0].time)`), confirmando que no se altera la dimensión de las muestras.
- **Tamaño de los mini-batches:** Exactamente igual a `batch_size` (8 o 16), indicando que el relleno funciona correctamente.
- **Relleno:** Se realiza con `xr.zeros_like`, manteniendo la estructura de las muestras originales.

**Conclusión:** La función `pad_batches` garantiza mini-batches uniformes, cumpliendo con los requisitos esperados.

## 5.Creación de una Clase Generadora de Batches

Para organizar y reutilizar nuestro código de manera eficiente, encapsularemos toda la funcionalidad anterior en una clase BatchGenerator. Esta clase permitirá generar batches de manera sencilla, con opciones para barajar y aplicar padding

In [ ]:
class BatchGenerator:
    def __init__(self, dataset: xr.Dataset, sample_size: int, batch_size: int, shuffle: bool = True, padding: bool = True):
        """
        Inicializa el BatchGenerator.

        Args:
            dataset (xr.Dataset): El conjunto de datos.
            sample_size (int): Tamaño de cada ventana deslizante.
            batch_size (int): Tamaño de cada batch.
            shuffle (bool): Si se deben barajar las muestras.
            padding (bool): Si se debe aplicar padding al último batch.
        """
        self.dataset = dataset
        self.sample_size = sample_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.padding = padding
        self.samples = self.generate_sliding_windows()
        if self.shuffle:
            self.samples = self.shuffle_samples()
        self.batches = self.generate_batches()
        if self.padding:
            self.batches = self.pad_batches()

    def generate_sliding_windows(self):
        """
        Genera ventanas deslizantes sobre el eje temporal del dataset.

        Returns:
            list: Lista de muestras generadas mediante ventanas deslizantes.
        """
        window_size = self.sample_size
        windows = [slice(i, i + window_size) for i in range(0, len(self.dataset.time) - window_size + 1)]
        samples = [self.dataset.isel(time=w) for w in windows]
        return samples

    def shuffle_samples(self):
        """
        Baraja las muestras generadas.

        Returns:
            list: Lista de muestras barajadas.
        """
        samples_copy = self.samples.copy()
        np.random.shuffle(samples_copy)
        return samples_copy

    def generate_batches(self):
        """
        Divide las muestras en batches.

        Returns:
            list: Lista de batches.
        """
        batches = [self.samples[i:i + self.batch_size] for i in range(0, len(self.samples), self.batch_size)]
        return batches

    def pad_batches(self):
        """
        Rellena el último batch con muestras vacías para que tenga el mismo tamaño que los demás.

        Returns:
            list: Lista de batches con el último batch rellenado si es necesario.
        """
        batches = self.batches.copy()
        if len(batches[-1]) < self.batch_size:
            last_batch = batches[-1]
            while len(last_batch) < self.batch_size:
                empty_sample = xr.zeros_like(last_batch[0])
                last_batch.append(empty_sample)
            batches[-1] = last_batch
        return batches

    def __iter__(self):
        """
        Iterador que genera batches de datos.

        Yields:
            list: Batch de datos.
        """
        for batch in self.batches:
            yield batch


In [48]:
# Supongamos que tenemos un dataset llamado 'dataset', y queremos generar batches.
sample_size = 3
batch_size = 8
batch_generator = BatchGenerator(dataset, sample_size, batch_size, shuffle=True, padding=True)

# Verificar el número de batches generados
print(f"Número de batches: {len(batch_generator.batches)}")
print(f"Tamaño de cada batch: {[len(batch) for batch in batch_generator.batches]}")


Número de batches: 320
Tamaño de cada batch: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 

Número de batches: El dataset generó correctamente 320 batches, confirmando que la función divide los datos en mini-batches de manera eficiente.

Tamaño de cada batch: Todos los batches tienen un tamaño uniforme de 8, lo cual cumple con el tamaño definido por batch_size.

In [33]:
# Verificar el tamaño de las muestras dentro de los batches
print(f"Tamaño de cada sample: {[len(batch[0].time) for batch in padded_batches]}")

Tamaño de cada sample: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


Tamaño de cada muestra dentro de los batches: Todas las muestras tienen un tamaño uniforme de 3, que coincide con el valor de sample_size especificado.

In [34]:
# Imprimir el último batch para verificar si fue rellenado
print("Último batch rellenado:", padded_batches[-1][-1])

Último batch rellenado: <xarray.Dataset> Size: 38MB
Dimensions:    (time: 3, depth: 49, latitude: 180, longitude: 180)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 24B 2014-02-03 2014-02-04 2014-02-05
Data variables:
    thetao     (time, depth, latitude, longitude) float64 38MB dask.array<chunksize=(3, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    t

In [35]:
# Probar con shuffle desactivado
batch_generator = BatchGenerator(dataset, sample_size, batch_size, shuffle=False, padding=True)
print("Muestras sin barajar (shuffle=False):", padded_batches[0][1])

Muestras sin barajar (shuffle=False): <xarray.Dataset> Size: 38MB
Dimensions:    (depth: 49, latitude: 180, longitude: 180, time: 3)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 24B 2015-01-02 2015-01-03 2015-01-04
Data variables:
    thetao     (time, depth, latitude, longitude) float64 38MB dask.array<chunksize=(3, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GL

In [36]:
print("Muestras sin barajar (shuffle=False):", padded_batches[0][0])

Muestras sin barajar (shuffle=False): <xarray.Dataset> Size: 38MB
Dimensions:    (depth: 49, latitude: 180, longitude: 180, time: 3)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 24B 2016-03-05 2016-03-06 2016-03-07
Data variables:
    thetao     (time, depth, latitude, longitude) float64 38MB dask.array<chunksize=(3, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GL

In [37]:
# Probar con tamaño de ventana diferente
sample_size = 4
batch_generator = BatchGenerator(dataset, sample_size, batch_size, shuffle=True, padding=True)
print(f"Muestras con tamaño de ventana 4:", batch_generator.batches[0][0])


Muestras con tamaño de ventana 4: <xarray.Dataset> Size: 51MB
Dimensions:    (depth: 49, latitude: 180, longitude: 180, time: 4)
Coordinates:
  * depth      (depth) float32 196B 0.494 1.541 2.646 ... 4.833e+03 5.275e+03
  * latitude   (latitude) float32 720B 19.58 19.67 19.75 ... 34.33 34.42 34.5
  * longitude  (longitude) float32 720B -20.92 -20.83 -20.75 ... -6.083 -6.0
  * time       (time) datetime64[ns] 32B 2015-08-12 2015-08-13 ... 2015-08-15
Data variables:
    thetao     (time, depth, latitude, longitude) float64 51MB dask.array<chunksize=(4, 49, 180, 180), meta=np.ndarray>
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GL